In [1]:
# importing necessary libraries
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import requests

In [2]:
# access database file from github repository through url requests
url = "https://github.com/Capricorn15/chinook-sales-analysis/raw/main/chinook.db"
response = requests.get(url)

with open("chinook.db", "wb") as f:
    f.write(response.content)
chinook_db = sqlite3.connect("chinook.db")

In [3]:
# define a function to read queries from database 
def query_df(query):
    return pd.read_sql_query(query, chinook_db)

In [4]:
# transaction query
query = '''
select
	inv.InvoiceId, 
    GROUP_CONCAT(tr.Name, ', ') Purchased_Tracks

from invoice_items it
left join tracks tr on it.TrackId = tr.TrackId
left join invoices inv on it.InvoiceId = inv.InvoiceId

group by inv.InvoiceId
order by inv.InvoiceId
'''
# store transaction query output dataframe in the variable below trxn_df
trxn_df = query_df(query)
trxn_df.head()

,InvoiceId,Purchased_Tracks
0,1,"Balls to the Wall, Restless and Wild"
1,2,"Put The Finger On You, Inject The Venom, Evil ..."
2,3,"Dog Eat Dog, Overdose, Love In An Elevator, Ja..."
3,4,"Right Through You, Not The Doctor, Bleed The F..."
4,5,"Your Time Has Come, Dandelion, Rock 'N' Roll M..."


In [5]:
# check for missing row values
trxn_df.isnull().sum()

InvoiceId           0
Purchased_Tracks    0
dtype: int64

In [6]:
trxn_df['Purchased_Tracks'] = trxn_df['Purchased_Tracks'].apply(lambda x: x.split(', '))

In [7]:
trxn_df

,InvoiceId,Purchased_Tracks
0,1,"[Balls to the Wall, Restless and Wild]"
1,2,"[Put The Finger On You, Inject The Venom, Evil..."
2,3,"[Dog Eat Dog, Overdose, Love In An Elevator, J..."
3,4,"[Right Through You, Not The Doctor, Bleed The ..."
4,5,"[Your Time Has Come, Dandelion, Rock 'N' Roll ..."
...,...,...
407,408,"[Bass Trap, Everlasting Love, Walk To The Wate..."
408,409,"[Trash, Trampoline And The Party Girl, Love An..."
409,410,"[Desire, Pride (In The Name Of Love), The Star..."
410,411,"[Higher Ground, Secrets, Eruption, Dreams, Eru..."


In [8]:
te = TransactionEncoder()
te_ary = te.fit(trxn_df['Purchased_Tracks']).transform(trxn_df['Purchased_Tracks'])
trxn_df_En = pd.DataFrame(te_ary, columns=te.columns_)

In [9]:
trxn_df_En

,"""?""","""Jupiter"": IV. Molto allegro","""Moonlight"": I. Adagio sostenuto",#9 Dream,'Round Midnight,(Anesthesia) Pulling Teeth,(White Man) In Hammersmith Palais,(Wish I Could) Hideaway,...And Found,...And Justice For All,...,and Future,the Bringer of Jollity,"the Secrets of Our Hearts""",Às Vezes,Água E Fogo,Água de Beber,Álibi,É Fogo,Étude 1,Óculos
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
408,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
409,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
410,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:
# Applying apriori principle 
frequent_itemsets = apriori(trxn_df_En, min_support=0.01, use_colnames=True)

# Generate association rules with minimum confidence
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.8)

# Display the top association rules
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
